In [1]:
import substra
import sh
import pathlib
import zipfile

In [2]:
client = substra.Client(backend_type="docker")
permissions = substra.schemas.Permissions(public=True, authorized_ids=[])

In [3]:
assets_directory = pathlib.Path("my_substra_files")

ALGO_TRAIN_DOCKERFILE_FILES = [
    assets_directory / "test.R",
    assets_directory / "python_wrapper.py",
    assets_directory / "Dockerfile",
]

train_archive_path = assets_directory / "algo.zip"
with zipfile.ZipFile(train_archive_path, "w") as z:
    for filepath in ALGO_TRAIN_DOCKERFILE_FILES:
        z.write(filepath, arcname=filepath.name)


train_algo_outputs = [substra.schemas.AlgoOutputSpec(identifier="model", kind=substra.schemas.AssetKind.model, multiple=False)]

train_algo = substra.schemas.AlgoSpec(
    name="Run R script",
    inputs=[],
    outputs=train_algo_outputs,
    description=assets_directory / "description.md",
    file=train_archive_path,
    permissions=permissions,
)


train_algo_key = client.add_algo(train_algo)

print(f"Train algo key {train_algo_key}")

Train algo key 4572a9a3-1920-4d4e-a7f1-4028ba569062


In [4]:
train_task = substra.schemas.TaskSpec(
    algo_key=train_algo_key,
    inputs=[],
    outputs={"model": substra.schemas.ComputeTaskOutputSpec(permissions=permissions)},
    worker=client.organization_info().organization_id,
)

train_task_key = client.add_task(train_task)

print(f"Train task key {train_task_key}")

Train task key 3db08d66-fe37-4438-9ef7-6bea7baa86d0


In [5]:
task = client.get_task(train_task_key)
print(task.status)

Status.done


In [6]:
import pickle
with open (client.download_model_from_task(train_task_key, "model", "."), 'rb') as f:
    print(pickle.load(f))

42
